In [4]:
import sys
import os
os.chdir("/home/jonfrey/ASL")
sys.path.append("""/home/jonfrey/ASL/src/""")
sys.path.append("""/home/jonfrey/ASL/src/pseudo_label""")

import numpy as np
from visu import Visualizer
import imageio


# STD
import os
import copy

# MISC
import numpy as np
import torch
import imageio
import cv2
from PIL import Image

# matplotlib
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.backends.backend_agg import FigureCanvasAgg
import matplotlib.patches as patches

from PIL import Image
import cv2
import torch
from torchvision import transforms as tf
from utils_asl import LabelLoaderAuto
from pathlib import Path
lla = LabelLoaderAuto(root_scannet="/home/jonfrey/Datasets/scannet")

In [16]:
# LOAD DATA

visu = Visualizer("/home/jonfrey/Documents/master_thesis/Report/images/regu", logger=None, epoch=0, store=False, num_classes=41)

def get_labels(p,sub=1,la="png"):
    p = [str(s) for s in Path(p).rglob(f'*.{la}') if str(s).find("_.png") == -1]
    
    p.sort(
        key=lambda x: int(x.split("/")[-1][:-4])
    )
    
    return p[::sub]

gt_p = get_labels("/home/jonfrey/Datasets/scannet/scans/scene0000_00/label-filt",sub=10)
img_p = get_labels("/home/jonfrey/Datasets/scannet/scans/scene0000_00/color",sub=10, la="jpg")
pred_p = get_labels("/home/jonfrey/Datasets/labels_generated/create_labels_from_pretrained/scans/scene0000_00/create_labels_from_pretrained")
pseudo_p = get_labels("/home/jonfrey/Datasets/labels_generated/52/scans/scene0000_00/52")
retrained_p = get_labels("//home/jonfrey/Datasets/labels_generated/labels_pretrain_retrain_52_05/scans/scene0000_00/labels_pretrain_retrain_52_05")

paths = [ gt_p , pred_p, pseudo_p,  retrained_p]
names = [ "gt", "pred", "ray_traced", "retrained"]

In [19]:
# CREATE VIDEO
import cv2

W, H = 1296, 968
from cv2 import VideoWriter, VideoWriter_fourcc
import cv2
FPS = 5

visu = Visualizer("/home/jonfrey/Documents/hypersim_examples/", logger=None, epoch=0, store=False, num_classes=41)
paths = [gt_p, pred_p, pseudo_p,  retrained_p]

for path, name in zip( paths, names):
    fourcc2 = VideoWriter_fourcc("M", "J", "P","G")
    video2 = VideoWriter(f'/home/jonfrey/Videos/{name}.avi', fourcc2, float(FPS), (W,H))

    for i in  range( 0,len( path  ),1 ):
        visu.epoch = i 
        label = lla.get( path[i])[0]
        img = imageio.imread(img_p[i])
        
        dra =  visu.plot_detectron( img, label , text_off =True, 
                                  jupyter=False,  draw_bound=True,
                                  store=False)
        
        dra_img = Image.fromarray(dra )
        r, g, b = dra_img.split()
        dra_img = np.array( Image.merge("RGB", (b, g, r)) )
        video2.write(dra_img)

    video2.release()